In [ ]:
#Algorithm:
#pick all the classes in clubbed and unclubbed manner and add count threshold to remove the classes with numbers
#keep threshold as 5 for clubbed and now put them in unclubbed dictionary and now filter unclubbed ones
#check the number of points in that manner
#ask for file location and create and put data there.
#random sampling algorithm to proceed with the dataset format.

In [1]:
#read the types in dictionary
types_location = "./types.txt"
type2id = {}
id2type = {}
with open(types_location) as f:
    lines = f.readlines()
    for i, line in enumerate(lines):
        line = line.strip().lower()
        type2id[line] = i
        id2type[i]    = line

In [2]:
train_data_locations     = ["./train"]
complete_data_locations = ["./train", "./dev", "./test"]

In [4]:
import os
import sys
def listdir(d):
    if not os.path.isdir(d):
        return [d]
    else:
        result=[]
        for item in os.listdir(d):
            result.extend(listdir((d+'/'+item)))
        return result
    
def get_files(list_of_dirs):
    result = []
    for d in list_of_dirs:
        result.extend(listdir(d))
    return result

In [5]:
from tqdm import tqdm
import json
import operator


# def read_data(files):
#     types_count = {}
#     for f in files:
#         for line in open(f, 'r'):
#             line = json.loads(line)
#             ystr = line['y_str']
#             for y in ystr:
#                 if not y in types_count:
#                     types_count[y] = 0
#                 types_count[y] = types_count[y]+1
#     return types_count

def type_list_to_id(ll):
    global type2id
    id_l = [str(type2id[l]) for l in ll]
    id_l.sort()
    return " ".join(id_l)

def id_str_to_type(id_str):
    global id2type
    ids = id_str.split(" ")
    return "/".join([id2type[int(id)] for id in ids])

def get_type_id(ll):
    ll.sort()
    return " ".join(ll)

#consider each class independent
def get_types_stats(files, type="clubbed"):
    types_count = {}
    for json_file in tqdm(files):
        for json_point in open(json_file, 'r'):
            data_point = json.loads(json_point)
            ystr = data_point['y_str']
            ystr.sort()
            k = "/".join(ystr)
            k = [k]
            if not type=="clubbed":
                k = ystr
            for key in k:
                if not key in types_count:
                    types_count[key] = 0
                types_count[key] = types_count[key]+1
    return types_count

In [6]:
import json
#write the dictionary for analysis
def write_dict(dictionary, output):
    sorted_d  = sorted(dictionary.items(), key=operator.itemgetter(0))
    types_str = [k+"\t"+str(v) for (k, v) in sorted_d]
    types_str = "\n".join(types_str)
    with open(output, "w") as f:
        f.write(types_str)
    print("written")

In [7]:
train_data_locations     = ["./train"]

In [8]:
train_files         = get_files(train_data_locations)
clubbed_types_count = get_types_stats(train_files)

100%|██████████| 23/23 [11:11<00:00, 67.49s/it]


In [21]:
#do filtering of the content of clubbed dictionary
print(len(clubbed_types_count))
del_count=0
for k, v in clubbed_types_count.items():
    if v<10:
        del clubbed_types_count[k]
        del_count+=1
print("Total number of keys deleted..." + str(del_count))
print("Total number of remaining keys..." + str(len(clubbed_types_count)))

71411
Total number of keys deleted...30687
Total number of remaining keys...40724


In [30]:
#get the concrete type of specific type
def get_concrete_type(t):
    loc = t.find("/")
    if loc==-1:
        return t
    else:
        return t[:loc]

    
def create_class_buckets(types_count, threshold=10):
    global type2id
    types_bucket = {}
    for k, v in types_count.items():
        concrete_type = get_concrete_type(k)
        if not concrete_type in types_bucket:
            types_bucket[concrete_type] = 0
        types_bucket[concrete_type]+=1
    for k, v in types_bucket.items():
        if v<threshold:
            del types_bucket[k]
    return types_bucket

In [26]:
#put values in actual dictionary and see the len of dictionaries and now you have list of different kinds of conflicts.

In [31]:
class_buckets = create_class_buckets(clubbed_types_count)

In [50]:
print("total number of class buckets are ")
print(len(class_buckets))
sorted_class_buckets = sorted(class_buckets.items(), key=operator.itemgetter(1))
sorted_class_buckets = sorted_class_buckets[:200]
final_class_buckets = set([a for a, b in sorted_class_buckets])

total number of class buckets are 
948


In [57]:
def get_general_to_specific_class_mapping(general_keys, specific_dict):
    output_mapping = {}
    classes_of_interest = []
    for k, v in specific_dict.items():
        concrete_type = get_concrete_type(k)
        if concrete_type in general_keys:
            if not concrete_type in output_mapping:
                output_mapping[concrete_type] = set([])
            output_mapping[concrete_type].add(k)
            classes_of_interest.append(k)
    return output_mapping, set(classes_of_interest)
gen_to_specific_type_dict, classes_of_interest = get_general_to_specific_class_mapping(final_class_buckets, clubbed_types_count)

In [61]:
train_data_locations     = ["./train"]
train_files         = get_files(train_data_locations)
def get_class_specific_training_data(classes_of_interest, files):
    class_specific_data = {}
    for json_file in tqdm(files):
        for json_point in open(json_file, 'r'):
            data_point = json.loads(json_point)
            ystr = data_point['y_str']
            ystr.sort()
            k = "/".join(ystr)
            if k in classes_of_interest:
                if not k in class_specific_data:
                    class_specific_data[k] = []
                class_specific_data[k].append(json_point)
    return class_specific_data

In [62]:
class_specific_data = get_class_specific_training_data(classes_of_interest, train_files)

100%|██████████| 23/23 [11:00<00:00, 61.52s/it]


In [67]:
total = 0
for k, v in class_specific_data.items():
    total+=len(v)
    
print("total number of data points")
print(total)

total number of data points
1269423


In [80]:
import os
import re
def get_file_name(k):
    return re.sub("/", "_", k)

In [90]:
# import shutil
# for k, v in tqdm(class_specific_data.items()):
#     concrete_type = get_concrete_type(k)
#     directory = "./support_sets" + concrete_type
#     try:
#         shutil.rmtree(directory)
#     except OSError as e:
#         print(directory)
# print("written everythng")

In [127]:
for k, v in tqdm(class_specific_data.items()):
    concrete_type = get_concrete_type(k)
    directory = os.path.join("./support_sets", concrete_type)
    if not os.path.exists(directory):
        os.makedirs(directory)
    v = "".join(v)
    fname = os.path.join(directory, get_file_name(k))
    with open(fname, "w") as f:
        f.write(v)

print("written everythng")

100%|██████████| 2166/2166 [00:07<00:00, 282.47it/s]

written everythng


In [ ]:
def listdir(d):
    if not os.path.isdir(d):
        return [d]
    else:
        result=[]
        for item in os.listdir(d):
            result.extend(listdir((d+'/'+item)))
        return result

In [95]:
def make_support_sets(dir_location):
    files = os.listdir(dir_location)
    print(files)

In [98]:
directories = os.listdir("./support_sets")
support_sets = []
for directory in directories:
    full_dir_path = os.path.join("./support_sets", directory)
    support_sets_per_directory = make_support_sets(full_dir_path)
    support_sets.extend(support_sets_per_directory)

['broadcasting', 'foot', 'mail', 'hunting', 'shot', 'father', 'campus', 'jet', 'cycling', 'concept', 'strike', 'drought', 'screen', 'daughter', 'broker', 'boxing', 'donor', 'chemistry', 'browser', 'friend', 'support', 'evaluation', 'delegation', 'statement', 'output', 'clan', 'jury', 'adjustment', 'side', 'status', 'engagement', 'lab', 'drinking', 'letter', 'geography', 'forest', 'round', 'burning', 'signal', 'disclosure', 'century', 'farming', 'acid', 'hormone', 'chess', 'cigarette', 'client', 'coup', 'flag', 'cocaine', 'grade', 'platform', 'comeback', 'painting', 'bottle', 'headline', 'bonds', 'copyright', 'commuter', 'ring', 'resource', 'affiliate', 'extradition', 'farmer', 'rebel', 'rain', 'incident', 'fitness', 'commerce', 'arbitration', 'initiative', 'hurricane', 'navy', 'lawmaker', 'entry', 'ambassador', 'batsman', 'basis', 'concession', 'impact', 'bomber', 'opera', 'creation', 'buyer', 'agriculture', 'pool', 'glass', 'discipline', 'item', 'alarm', 'evacuation', 'pact', 'parish'

In [143]:
def make_support_set(dir_location, total_support_sets=3, n_way=5, k_shot=1):
    data = []
    #read the content
    for f in os.listdir(dir_location):
        file_location = os.path.join(dir_location, f)
        with open(file_location, "r") as content:
            data.append(content.readlines())
    output = []
    total_classes = len(data)
    for _ in range(total_support_sets):
        ss = []
        x_hat = "null"
        #create one support set of specific out of these conflicts
        classes_to_pick = np.random.permutation(total_classes)[:n_way]
        test_point = np.random.permutation(len(classes_to_pick))[0]
        for i, c in enumerate(classes_to_pick):
            c_data = data[c]
            num_samples = len(c_data)
#             print(num_samples)
            samples = np.random.permutation(num_samples)[:k_shot+1]
#             print("samples are ")
#             print(samples)
            for s in samples[:-1]:
                ss.append(c_data[s])
            if i==test_point:
                x_hat = c_data[samples[-1]]
        support_set = "\n".join(ss)
        support_set+="\n---------------------------------------------------\n"
        support_set+=x_hat+"\n"
        support_set+="------------------------------------------------------\n\n"
        output.append(support_set)
    return output

In [146]:
directories = os.listdir("./support_sets")
support_sets = []
for directory in tqdm(directories):
    full_dir_path = os.path.join("./support_sets", directory)
    support_sets_per_directory = make_support_set(full_dir_path)
    support_sets.extend(support_sets_per_directory)
support_sets = "".join(support_sets)
with open("./all_support_sets.json", "w") as f:
    f.write(support_sets)
print("All the support sets are available")

100%|██████████| 200/200 [00:00<00:00, 239.31it/s]

All the support sets are available


In [ ]:
#function to perform this operation
def get_stats_data_frame(types_count):
    global type2id
    types_bucket = {k:[] for k  in type2id.keys()}
    min_bucket   = {k:0  for k  in type2id.keys()}
    max_bucket   = {k:0  for k  in type2id.keys()}
    total_bucket = {k:0  for k  in type2id.keys()}
    for k, v in types_count.items():
#         print(k)
        concrete_type = get_concrete_type(k)
        if not concrete_type in types_bucket:
            print("issue with the type")
            print(concrete_type)
            print(k)
            types_bucket[concrete_type]=[]
            min_bucket[concrete_type]=0
            max_bucket[concrete_type]=0
            total_bucket[concrete_type]=0
        l = types_bucket[concrete_type]
        l.append(k)
        types_bucket[concrete_type] = l 
        min_bucket[concrete_type]   = min(v, min_bucket[concrete_type]) if not min_bucket[concrete_type]==0 else v
        max_bucket[concrete_type]   = max(v, max_bucket[concrete_type])
        total_bucket[concrete_type] = total_bucket[concrete_type] + v
    pollution_per_class = {k:len(v) for k, v in types_bucket.items() if len(v)>0}
    types_bucket = {k:" && ".join(v) for k, v in types_bucket.items() if len(v)>0}
    min_bucket  =  {k:v for k, v in min_bucket.items() if k in types_bucket.keys()}
    max_bucket  =  {k:v for k, v in max_bucket.items() if k in types_bucket.keys()}
    total_bucket  =  {k:v for k, v in total_bucket.items() if k in types_bucket.keys()}
    print("processed everything")
    return types_bucket, pollution_per_class, min_bucket, max_bucket, total_bucket